Combinatorial Testing
=====================

Nowadays, software systems are complex and can have multiple possible configurations, for example, an application can have multiple target operating systems, execute in various types of hardware, and in multiple resolutions. But also those applications can have multiple states which they are in.
Those multiple parameters for a system cause different behaviors alone and when combined, so multiple combinations of a system’s parameters must be tested to achieve optimal test coverage, and catch errors that wouldn't be detected executing simple tests.

An example of an application that can have multiple possible configurations, is a mobile application, that can execute in multiple types of cellphone and in multiple possible states. A example of possible parameters {cite}`combinatorialexample` for a mobile application are:

| Parameter   | Options                       |
|-------------|-------------------------------|
| Orientation | portrait, landscape           |
| OS          | iOS, Android                  |
| Screen Size | 1080x1920, 750x1334, 720x1280 |

Using those parameters, we can calculate the list of possible parameter combinations multiplying the number of options for each parameter:

```
    Number of configuration combinations = 2 * 2 * 3 = 12 combinations of parameters
```

With a small quantity of parameters and options, this number is already big. But as the number of parameters and its possible values raise, the number of possible combinations rises exponentially, making it impossible to exhaustively test the software, given the time and budget constraints often existent on software projects.

A way to overcome those limitations is by using Combinatorial Testing, a method for software testing that for some input parameters, tests possible discrete combinations of those parameters, generating those combinations by systematically covering t-way interactions between parameters {cite}`combinatorialsurvey`, the "t" being called the degree of interaction, this set of combinations is called a *covering array*.

## Choosing a degree of interaction
Constructing a Combinatorial Testing suite, we have to specify the input parameters and values, but there’s an important parameter we must think about: the degree of interaction. A degree of interaction t means that we want to test the t-way interaction of our parameters, that is, we want to test all combinations of t parameters.	

Depending on the value chosen for "t", the process of generating the input can be more or less computationally complex but also can achieve more or less fault coverage. Increasing the value for "t" will increase the fault coverage, but also will increase the cost for generating and executing the tests, getting to a point that's almost no gain in coverage as the "t"  increases.

So, you would want to choose a degree of interaction that gives you an appropriate level of confidence but doesn't make the process of testing to costly. The creators of ACTS, a popular tool for combinatorial testing, have done research on the relation of the degree of interactions and software failures. They found that most of the bugs studied were caused by an interaction of at most 6 different parameters {cite}`actscombinatorial`:

```{figure} ../assets/interactions_chart.png
---
name: my-figure
---
Most failures are triggered by one or two parameters interacting, with progressively fewer by 3, 4, or more {cite}`actscombinatorial`.
```

## Hands On [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/damorimRG/practical_testing_book/blob/master/testgeneration/combinatorial.ipynb)
Now that you understand what combinatorial testing is, let’s see in practice how it is done.


Let’s take a looking at a important code that we want to test:

In [ ]:
%%file important_code.py
def important_function(pressure, volume, velocity, low_fuel):
    if pressure < 10:
        if volume > 300:
            if velocity == 5:
                do_something_bad()
        elif low_fuel:
            do_something_good()
    else:
        do_something_good()
 
def do_something_good():
    pass
 
def do_something_bad():
    raise Exception("A bug happened!")



We want to test important_function, which takes three integer parameters and a boolean. In this case, we want to test many combinations of parameters trying to be more confident that this code doesn't have a bug.

### Writing combinatorial tests
Now that we took a look in the code that we want to test, let's see how we can generate the combination of parameters that will spot the bug.

First, we need to install both pytest and covertable dependencies:

In [2]:
%%bash

pip install pytest
pip install covertable

Now that we have the dependencies installed, let's take a look in the file `test_parameterized.py`, which implements a test using those libraries.

In [ ]:
%%file test_parameterized.py
import pytest
from covertable import make
import important_code
 
@pytest.mark.parametrize(["pressure", "volume", "velocity", "low_fuel"],
    make([[5,10,15],
        [200, 300, 400],
        [1, 2, 3, 4, 5],
        [True, False]], length=3)
)
def test_important_function(pressure, volume, velocity, low_fuel):
    important_code.important_function(pressure, volume, velocity, low_fuel)

As you can see, `pytest` provides us the annotation `@pytest.mark.parametrize` that receives two arguments: an array of parameters names and a array of array of values for each of those parameters. The function below this annotation will be tested for each of the values specified in the second argument.



In the first argument, we need to provide the name of the parameters we want to test. In this case, as we want to test `important_function`, we need to pass `["pressure", "volume", "velocity", "low_fuel"]`.

In the second argument, we need to pass an array of array of values, that would be the combination of parameters given to `important_function`. To do that, we will be using `covertable` function called `make`. This function receives an array of array of values and return an array with the combination of them. To be more clear, as we passed `[5,10,15]` as the first element of the array, those are the possible values for the parameter `pressure`, `[200, 300, 400]` would be the possible values for the parameter `volume`, and so on.

It's important to notice that `make` also accepts some configuration. In the second argument, we set `length=3`. This is the degree of interaction that we teached earlier in this article. As we need the combination of `pressure`, `volume`, and `velocity` to catch the bug, our degree of interaction will be 3
(in a real-world scenario, though, we wouldn’t know when a bug happens. So, you would want to choose a degree of interaction that gives you an appropriate level of confidence, as we discussed).

Now that we have our test file, we just need to run pytest and see if there's a bug:

In [ ]:
%%bash

python -m pytest

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 45 items

sample_data/test_parameterized.py ...........F.......................... [ 84%]
.......                                                                  [100%]

=================================== FAILURES ===================================
____________________ test_important_function[5-400-5-False] ____________________

pressure = 5, volume = 400, velocity = 5, low_fuel = False

    @pytest.mark.parametrize(["pressure", "volume", "velocity", "low_fuel"],
        make([[5,10,15],
            [200, 300, 400],
            [1, 2, 3, 4, 5],
            [True, False]], length=3)
    )
    def test_important_function(pressure, volume, velocity, low_fuel):
>       important_function(pressure, volume, velocity, low_fuel)

sample_data/test_parameterized.py:27: 
_ _ _ _ _ _ _

When running this command, you will see the log of parameters used to test the function and spot the bug caused by the combination of values that satisfies `pressure < 10, volume > 300, and velocity == 5`.


## Exercise

Let's practice what you learned. Using pytest and covertable, try to make a combinatorial test that will cover all interaction between the passed parameters in this domain below.

In [ ]:
import pytest
from covertable import make

def get_orientation(phoneWidthPixels, phoneHeightPixels, operatingSystem):
  if phoneWidthPixels >= 320 and phoneWidthPixels <= 640 and phoneHeightPixels <= 1136:
    if operatingSystem == "android" or operatingSystem == "fuchsia":
        return "landscape"
    elif operatingSystem == "ios":
        return "portrait"
  elif phoneWidthPixels > 640 and phoneWidthPixels < 1242 and phoneHeightPixels > 1136 and phoneHeightPixels < 2208:
    if operatingSystem == "android" or operatingSystem == "fuchsia":
        return "portrait"
    elif operatingSystem == "ios":
        return "landscape"
  else:
    return "portrait"


## Testing

@pytest.mark.parametrize(["phoneWidthPixels", "phoneHeightPixels", "operatingSystem"],
    make([[], [], []])
)

def test_get_orientation(phoneWidthPixels, phoneHeightPixels, operatingSystem):
    get_orientation(phoneWidthPixels, phoneHeightPixels, operatingSystem)

## References

```{bibliography} ../zreferences.bib
    :style: alpha 
    :filter: docname in docnames
```